In [1]:
#Kurulum (Kütüphaneler)
!pip install -q pandas numpy sentence-transformers faiss-cpu tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 99.8 MB/s eta 0:00:00


In [3]:
#Importlar ve Genel Ayarlar
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import torch
from sentence_transformers import SentenceTransformer
import faiss

tqdm.pandas()
device = "cuda" if torch.cuda.is_available() else "cpu"
device


'cuda'

In [4]:
#Chunk Veri Setini Yükle
chunks_path = "https://raw.githubusercontent.com/guzelTemli/YapayZekaChatbot/refs/heads/main/bilgi_guvenligi_dataset/bilgi_guvenligi_chunks_v3_tr_FOR_RAG_C_FINAL.csv"

chunks_df = pd.read_csv(chunks_path)

print("Chunk satır sayısı:", len(chunks_df))
chunks_df.head()


Chunk satır sayısı: 247


,filename,source_url,section_label,section_index,chunk_index,chunk_text,chunk_text_tr
0,ISO-27001-Controls-List-Free-Download-pdf-.pdf,https://hightable.io/iso-27001-annex-a-control...,ISO_5.1,5.10,0,5.1 Policies for information security Informat...,5.1 Bilgi güvenliğine ilişkin politikalar Bilg...
1,ISO-27001-Controls-List-Free-Download-pdf-.pdf,https://hightable.io/iso-27001-annex-a-control...,ISO_5.10,5.10,0,5.10 Acceptable use of information and other a...,5.10 Bilginin ve diğer ilgili varlıkların kabu...
2,ISO-27001-Controls-List-Free-Download-pdf-.pdf,https://hightable.io/iso-27001-annex-a-control...,ISO_5.11,5.11,0,5.11 Return of Assets Personnel and other inte...,5.11 Varlıkların İadesi Personel ve diğer ilgi...
3,ISO-27001-Controls-List-Free-Download-pdf-.pdf,https://hightable.io/iso-27001-annex-a-control...,ISO_5.12,5.12,0,5.12 Classification Of Information Information...,"5.12 Bilgilerin Sınıflandırılması Bilgiler, ku..."
4,ISO-27001-Controls-List-Free-Download-pdf-.pdf,https://hightable.io/iso-27001-annex-a-control...,ISO_5.13,5.13,0,5.13 Labelling of Information An appropriate s...,5.13 Bilgilerin Etiketlenmesi Bilgi etiketleme...


In [5]:
#chunk_id ve Metin Alanını Hazırla

# Her satıra global id
chunks_df["chunk_id"] = chunks_df.index.astype(int)

# TR + EN metni birleştir
chunks_df["text"] = (
    chunks_df["chunk_text_tr"].fillna("")
    + "\n" +
    chunks_df["chunk_text"].fillna("")
).str.strip()

# Boş olmayanları kontrol edelim (ama şimdilik filtreleme yapmıyoruz)
print("Boş text sayısı:", (chunks_df["text"].str.len() == 0).sum())

chunks_df[["chunk_id", "filename", "section_label", "chunk_index", "text"]].head()


Boş text sayısı: 0


,chunk_id,filename,section_label,chunk_index,text
0,0,ISO-27001-Controls-List-Free-Download-pdf-.pdf,ISO_5.1,0,5.1 Bilgi güvenliğine ilişkin politikalar Bilg...
1,1,ISO-27001-Controls-List-Free-Download-pdf-.pdf,ISO_5.10,0,5.10 Bilginin ve diğer ilgili varlıkların kabu...
2,2,ISO-27001-Controls-List-Free-Download-pdf-.pdf,ISO_5.11,0,5.11 Varlıkların İadesi Personel ve diğer ilgi...
3,3,ISO-27001-Controls-List-Free-Download-pdf-.pdf,ISO_5.12,0,"5.12 Bilgilerin Sınıflandırılması Bilgiler, ku..."
4,4,ISO-27001-Controls-List-Free-Download-pdf-.pdf,ISO_5.13,0,5.13 Bilgilerin Etiketlenmesi Bilgi etiketleme...


In [6]:
#Huggingface'e bağlanma
from huggingface_hub import login
login()

In [7]:
#Embedding Modelini Yükle
model_name = "intfloat/multilingual-e5-large"
embed_model = SentenceTransformer(model_name, device=device)
embed_model.max_seq_length = 512

embed_model


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False, 'architecture': 'XLMRobertaModel'})
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [8]:
#Chunk’ları Embed Et ve FAISS Index Kur
texts = chunks_df["text"].tolist()
chunk_ids = chunks_df["chunk_id"].to_numpy()

# E5 için passage prefix'i
passages = [f"passage: {t}" for t in texts]

batch_size = 32
emb_list = []

for i in tqdm(range(0, len(passages), batch_size), desc="Chunk embedding"):
    batch = passages[i:i+batch_size]
    with torch.no_grad():
        emb = embed_model.encode(
            batch,
            show_progress_bar=False,
            convert_to_numpy=True,
            normalize_embeddings=True
        )
    emb_list.append(emb)

embeddings = np.vstack(emb_list)
dim = embeddings.shape[1]

print("Embedding shape:", embeddings.shape)

# FAISS index (cosine ≈ inner product)
index = faiss.IndexFlatIP(dim)
index.add(embeddings)

print("FAISS vektör sayısı:", index.ntotal)


Chunk embedding:   0%|          | 0/8 [00:00<?, ?it/s]

Embedding shape: (247, 1024)
FAISS vektör sayısı: 247


In [9]:
#search_chunks Fonksiyonu (RAG’in Kalbi yani retriver kısmı)

def search_chunks(query: str, top_k: int = 5):
    """
    Verilen soruya en çok benzeyen top_k chunk'ı getirir.
    Dönen her eleman:
        - score
        - chunk_id (global)
        - filename, section_label, chunk_index
        - text (TR+EN)
    """
    query_text = f"query: {query}"

    with torch.no_grad():
        q_emb = embed_model.encode(
            [query_text],
            convert_to_numpy=True,
            normalize_embeddings=True
        )

    scores, indices = index.search(q_emb, top_k)

    results = []
    for score, idx in zip(scores[0], indices[0]):
        cid = int(chunk_ids[idx])  # FAISS sırasından gerçek chunk_id
        row = chunks_df[chunks_df["chunk_id"] == cid].iloc[0]

        results.append({
            "score": float(score),
            "chunk_id": cid,
            "chunk_index": int(row["chunk_index"]),
            "section_label": row["section_label"],
            "filename": row["filename"],
            "source_url": row.get("source_url", None),
            "text": row["text"]
        })
    return results


In [10]:
#Test için
test_q = "Uzaktan çalışma sırasında hangi güvenlik önlemleri alınmalıdır?"
res = search_chunks(test_q, top_k=3)

for i, r in enumerate(res, 1):
    print("="*80)
    print(f"[{i}] score={r['score']:.3f} | chunk_id={r['chunk_id']} | {r['filename']} | {r['section_label']} | chunk_index={r['chunk_index']}")
    print(r["text"][:400], "...")


[1] score=0.895 | chunk_id=43 | ISO-27001-Controls-List-Free-Download-pdf-.pdf | ISO_6.7 | chunk_index=0
6.7 Uzaktan çalışma Personel, kuruluşun tesisleri dışında erişilen, işlenen veya saklanan bilgileri korumak için uzaktan çalışırken güvenlik önlemleri uygulanmalıdır.
6.7 Remote working Security measures should be implemented when personnel are working remotely to protect information accessed, processed or stored outside the organization’s premises. ...
[2] score=0.837 | chunk_id=55 | ISO-27001-Controls-List-Free-Download-pdf-.pdf | ISO_7.6 | chunk_index=0
7.6 Güvenli alanlarda çalışma Güvenli alanlarda çalışmaya yönelik güvenlik önlemleri tasarlanmalı ve uygulanmalıdır.
7.6 Working in secure areas Security measures for working in secure areas should be designed and implemented. ...
[3] score=0.837 | chunk_id=188 | SANS_Perimeter_Network_Access_Management_Policy_April2025.pdf | Safeguards | chunk_index=1
ağ bağlantılarının filtrelenmesi, yalnızca yetkili kullanıcıların bir kuruluşun

In [11]:
#QA Master Dosyasını Yükle
qa_path = "https://raw.githubusercontent.com/guzelTemli/YapayZekaChatbot/refs/heads/main/bilgi_guvenligi_dataset/bilgi_guvenligi_qa_master.csv"
qa_df = pd.read_csv(qa_path)

print("Toplam QA satırı:", len(qa_df))
qa_df.head()


Toplam QA satırı: 614


,question_id,question,answer,question_type,source_filenames,source_sections,source_chunk_ids,is_impossible,dataset_split
0,QA0001,What are some examples of activities that are ...,The policy prohibits violations of intellectua...,simple,SANS_Acceptable_Use_Standard_April2025.pdf,Safeguards,3,False,train
1,QA0002,What are the potential consequences for indivi...,Non-compliance with this policy may result in ...,simple,SANS_Safeguard_Implementation_Management_Polic...,Safeguards,2,False,train
2,QA0003,"According to the policy, what specific securit...","To ensure the security of wireless networks, a...",complex,SANS_Internal_Network_Access_Management_Policy...,Safeguards|Safeguards,0|1,False,train
3,QA0004,What are the security implications of storing ...,This information is not specified in the polic...,negative,SANS_Database_Credentials_Standard_April2025.pdf,full,3,True,train
4,QA0005,Which security controls are required to be imp...,The policy requires the implementation of a cl...,simple,SANS_Cloud_Service_Provider_Management_Policy_...,Safeguards,1,False,train


In [12]:
#(filename, section_label, chunk_index) → chunk_id Sözlüğünü Kur

# (filename, section_label, chunk_index) -> chunk_id sözlüğü
key_to_chunkid = {}

for _, r in chunks_df.iterrows():
    key = (str(r["filename"]), str(r["section_label"]), int(r["chunk_index"]))
    key_to_chunkid[key] = int(r["chunk_id"])

len(key_to_chunkid)


247

In [13]:
#QA Satırından Gold Chunk ID’leri Çıkaran Fonksiyon

def get_gold_chunk_ids(row):
    """
    QA satırından gerçek global chunk_id listesini üretir.
    (filename, section_label, chunk_index) üçlüsünü kullanır.
    Bir soru birden fazla chunk'a referans verebilir.
    """
    fnames   = str(row["source_filenames"]).split("|")
    sections = str(row["source_sections"]).split("|")
    c_idxs   = str(row["source_chunk_ids"]).split("|")

    gold_ids = []
    for f, s, c in zip(fnames, sections, c_idxs):
        f = f.strip()
        s = s.strip()
        c = c.strip()
        if not c:
            continue
        try:
            key = (f, s, int(c))
        except ValueError:
            continue
        cid = key_to_chunkid.get(key)
        if cid is not None:
            gold_ids.append(cid)
    return gold_ids


In [14]:
#Test için

row0 = qa_df.iloc[0]
print("Soru:", row0["question"])
print("Kaynak triple:", row0["source_filenames"], "|", row0["source_sections"], "|", row0["source_chunk_ids"])
print("Gold chunk_ids:", get_gold_chunk_ids(row0))


Soru: What are some examples of activities that are strictly prohibited according to the policy?
Kaynak triple: SANS_Acceptable_Use_Standard_April2025.pdf | Safeguards | 3
Gold chunk_ids: [100]


In [15]:
#Recall@k Hesaplama Fonksiyonu

def evaluate_recall_at_k(qa_subset: pd.DataFrame, k: int = 5):
    """
    Verilen QA subset için Recall@k hesaplar:
    - Her soru için search_chunks ile top-k sonuç alınır.
    - Eğer top-k içinde en az bir doğru chunk_id varsa -> hit = 1
      aksi halde -> 0
    """
    hits = []

    for _, row in tqdm(qa_subset.iterrows(), total=len(qa_subset), desc=f"Eval @ {k}"):
        gold_ids = get_gold_chunk_ids(row)
        if not gold_ids:
            # Bu soru için tanımlı doğru chunk yoksa atla
            continue

        results = search_chunks(row["question"], top_k=k)
        retrieved_ids = [r["chunk_id"] for r in results]

        hit = int(any(cid in retrieved_ids for cid in gold_ids))
        hits.append(hit)

    if not hits:
        return float("nan")

    recall = sum(hits) / len(hits)
    return recall


In [16]:
# Tüm Set İçin Recall@k Sonuçlarını Hesapla

def run_full_evaluation(qa_df: pd.DataFrame):
    # İmkansız olmayan sorular (simple + complex)
    non_impossible = qa_df[qa_df["is_impossible"] == False].copy()

    simple_df  = non_impossible[non_impossible["question_type"] == "simple"]
    complex_df = non_impossible[non_impossible["question_type"] == "complex"]

    results = {}
    for k in [3, 5, 10]:
        results[f"all@{k}"]     = evaluate_recall_at_k(non_impossible, k=k)
        results[f"simple@{k}"]  = evaluate_recall_at_k(simple_df, k=k)
        results[f"complex@{k}"] = evaluate_recall_at_k(complex_df, k=k)

    return results

metrics = run_full_evaluation(qa_df)
metrics


Eval @ 3:   0%|          | 0/554 [00:00<?, ?it/s]

Eval @ 3:   0%|          | 0/383 [00:00<?, ?it/s]

Eval @ 3:   0%|          | 0/171 [00:00<?, ?it/s]

Eval @ 5:   0%|          | 0/554 [00:00<?, ?it/s]

Eval @ 5:   0%|          | 0/383 [00:00<?, ?it/s]

Eval @ 5:   0%|          | 0/171 [00:00<?, ?it/s]

Eval @ 10:   0%|          | 0/554 [00:00<?, ?it/s]

Eval @ 10:   0%|          | 0/383 [00:00<?, ?it/s]

Eval @ 10:   0%|          | 0/171 [00:00<?, ?it/s]

{'all@3': 0.7256317689530686,
 'simple@3': 0.7154046997389034,
 'complex@3': 0.7485380116959064,
 'all@5': 0.7906137184115524,
 'simple@5': 0.7728459530026109,
 'complex@5': 0.8304093567251462,
 'all@10': 0.8501805054151624,
 'simple@10': 0.8433420365535248,
 'complex@10': 0.8654970760233918}

In [22]:
import os
import numpy as np

# 1) KAYIT KLASÖRÜ
save_dir = "/content"
os.makedirs(save_dir, exist_ok=True)

# 2) FAISS INDEX'İ YAZ
import faiss
faiss.write_index(index, os.path.join(save_dir, "faiss_index.bin"))

# 3) FAISS SIRASI -> CHUNK_ID MAPPING
np.save(os.path.join(save_dir, "chunk_ids.npy"), chunk_ids)

# 4) CHUNK META BİLGİSİ (metin ve mapping için)
cols_to_keep = ["chunk_id", "filename", "section_label", "chunk_index", "text"]
chunks_df[cols_to_keep].to_csv(os.path.join(save_dir, "chunks_meta.csv"), index=False)

# 5) KONTROL
!ls -lh /content


total 1.4M
-rw-r--r-- 1 root root 2.1K Nov 30 22:40 chunk_ids.npy
-rw-r--r-- 1 root root 418K Nov 30 22:40 chunks_meta.csv
-rw-r--r-- 1 root root 989K Nov 30 22:40 faiss_index.bin
drwxr-xr-x 1 root root 4.0K Nov 20 14:30 sample_data


In [23]:
from google.colab import files
files.download("/content/faiss_index.bin")
files.download("/content/chunk_ids.npy")
files.download("/content/chunks_meta.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>